In [1]:
mes = "06"

In [2]:
#manipulação de dados
import pandas as pd
import numpy as np

#automação
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#web scraping
import requests
from bs4 import BeautifulSoup

#string e data
import unidecode

#tempo
import time
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings #Warning inuteis
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [3]:
dispen = pd.read_excel('dispen.xlsx')
dispen

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO
0,[30/06/2023 06:27:50] Yoko Hayashi: 30/06 Itai...,30/06/2023,YOKO HAYASHI,30/06 ITAIM YOKO ( DISPENSADOPOR MOTIVO DE SA...,PROB,30/06/2023,ITAIM,1,DISPENSADO,NaN,NaN,NaN,semana,NaN,NaN,NaN,NaN


In [4]:
dispen['controle'] = None

In [5]:
#tempo
import time
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import (visibility_of, 
                                                           staleness_of, 
                                                           invisibility_of_element)


browser = webdriver.Firefox()

In [6]:
browser.get('https://www.pegaplantao.com.br/login')
sleep(1)
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_UserName"]').send_keys('gabriel_s_anjos@yahoo.com') #e-mail
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_Password"]').send_keys('Gabriel2022!') #senha
browser.find_element("xpath", '//*[@id="MainContent_LoginUser_btnLogin"]').click() #login

In [7]:
browser.get('https://www.pegaplantao.com.br/EscalaMensal')

In [8]:
#click setor/profissional
browser.find_element("xpath", '//*[@id="s2id_autogen1"]/a/span[1]').click()
#click [profissional]
browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li[2]/div').click()

In [9]:
browser.find_element("xpath", '//*[@id="s2id_ddlProfessionals"]/a/span[1]').click()

In [10]:
browser.find_element("xpath", '/html/body/div[6]/div/input').send_keys(dispen['nome'][0])
browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li/div').click()

#esperando carregamento do site
WebDriverWait(browser, 30).until_not(visibility_of(browser.find_element("xpath", '//*[@id="loader"]')))

False

In [11]:
html = browser.page_source
soup = BeautifulSoup(html, 'lxml')
tab = soup.find('table')
tbody = tab.tbody
td = tbody.find_all('td')

In [12]:
##unwanted = tbody.find_all('td', class_="another-month")

In [13]:
##for i in unwanted:
  #  i.extract()

In [14]:
# cria lista dos dias dos mes, com 00 no lugar de dias de outros meses
dias = []
for i in td:
    try:
        if i['class'] == ['another-month']:
            dias.append('00')
    except:
        spans = i.find('span', {'class':"day-number"}).text
        dias.append(spans)

In [15]:
#cria um df com a lista dias na posição certa 
tabela = pd.DataFrame(np.array(dias).reshape(5,7), columns=['1','2','3','4','5','6','7'], index=['1','2','3','4','5'])

In [16]:
tabela

,1,2,3,4,5,6,7
1,00,00,00,01,02,03,04
2,05,06,07,08,09,10,11
3,12,13,14,15,16,17,18
4,19,20,21,22,23,24,25
5,26,27,28,29,30,00,00


In [17]:
def existencia(caminho):
    try:
        browser.find_element("xpath", caminho)
        return True
    except NoSuchElementException:
        return False

In [18]:
apagar = [dispen['data'][0][:2]]

In [19]:
apagar

['30']

In [20]:
dispen

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO
0,[30/06/2023 06:27:50] Yoko Hayashi: 30/06 Itai...,30/06/2023,YOKO HAYASHI,30/06 ITAIM YOKO ( DISPENSADOPOR MOTIVO DE SA...,None,30/06/2023,ITAIM,1,DISPENSADO,NaN,NaN,NaN,semana,NaN,NaN,NaN,NaN


In [21]:
count = 0
for cada in apagar:
    ##tr e td começam em 1 , índices e colunas em 0
    tr_ind = str(np.where(tabela==cada)[0]+1)
    td_ind = str(np.where(tabela==cada)[1]+1)
    while(existencia('/html/body/form/main/div[2]/div[2]/table/tbody/tr'+tr_ind+'/td'+td_ind+'/ul[1]/li')):
        elemento = browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/table/tbody/tr'+tr_ind+'/td'+td_ind+'/ul[1]/li')
        elemento.click()
        sleep(4)
        botao = browser.find_element(By.CSS_SELECTOR, 'button.pp-c-btn:nth-child(3)')
        botao.click()
        sleep(4)
        botao2 = browser.find_element(By.CSS_SELECTOR, 'button.btnpp-info:nth-child(4)')
        botao2.click()
        sleep(4)
        print(str(count)+': apagado'+' dispensa de '+dispen['nome'][0]+" em "+dispen['data'][0])
        


In [22]:
if existencia('/html/body/form/main/div[2]/div[2]/table/tbody/tr'+tr_ind+'/td'+td_ind+'/ul[1]/li') is False:
    dispen['controle'][0] = 'CERTO'
    count += 1
else: 
    dispen['controle'][0] = 'PROB'
    count += 1
    print(str(count)+": deu erro de "+dispen['nome'][0]+" em "+dispen['data'][0])

In [23]:
count

1

In [24]:
for plt in range(count, len(dispen)):
    html = browser.find_element(By.TAG_NAME, 'html')
    html.send_keys(Keys.HOME)
    WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="s2id_ddlProfessionals"]/a/span[1]')))
    sleep(0.5)
    browser.find_element("xpath", '//*[@id="s2id_ddlProfessionals"]/a/span[1]').click()
    browser.find_element("xpath", '/html/body/div[8]/div/input').send_keys(dispen['nome'][plt])
    browser.find_element("xpath", '//*[@id="select2-drop"]/ul/li/div').click()
    sleep(4)
    apagar = dispen['data'][plt][:2]
    
    
    html = browser.page_source
    soup = BeautifulSoup(html, 'lxml')
    tab = soup.find('table')
    tbody = tab.tbody
    td = tbody.find_all('td')
    
    dias = []
    for i in td:
        try:
            if i['class'] == ['another-month']:
                dias.append('00')
        except:
            spans = i.find('span', {'class':"day-number"}).text
            dias.append(spans)
            
    tabela = pd.DataFrame(np.array(dias).reshape(5,7), columns=['1','2','3','4','5','6','7'], index=['1','2','3','4','5'])
    
    ##tr e td começam em 1 , índices e colunas em 0
    tr_ind = str(np.where(tabela==apagar)[0]+1)
    td_ind = str(np.where(tabela==apagar)[1]+1)
    while(existencia('/html/body/form/main/div[2]/div[2]/table/tbody/tr'+tr_ind+'/td'+td_ind+'/ul[1]/li')):
        elemento = browser.find_element("xpath", '/html/body/form/main/div[2]/div[2]/table/tbody/tr'+tr_ind+'/td'+td_ind+'/ul[1]/li')
        elemento.click()
        sleep(4)
        botao = browser.find_element(By.CSS_SELECTOR, 'button.pp-c-btn:nth-child(3)')
        botao.click()
        sleep(4)
        botao2 = browser.find_element(By.CSS_SELECTOR, 'button.btnpp-info:nth-child(4)')
        botao2.click()
        sleep(4)
        print(str(count)+': apagado'+' dispensa de '+dispen['nome'][plt]+" em "+dispen['data'][plt])
    
    if existencia('/html/body/form/main/div[2]/div[2]/table/tbody/tr'+tr_ind+'/td'+td_ind+'/ul[1]/li') is False:
        dispen['controle'][plt] = 'CERTO'
        count += 1
    else: 
        dispen['controle'][plt] = 'PROB'
        count += 1
        print(str(count)+": deu erro de "+dispen['nome'][plt]+" em "+dispen['data'][plt])

In [25]:
count

1

In [26]:
len(dispen)

1

In [27]:
dispen

,chat,dia,nome,mensagem,controle,data,unidade,Q_unidades,sit,extra,tipo,hora,sem_ou_fds,hora2,ENTRADA,SAIDA,PERIODO
0,[30/06/2023 06:27:50] Yoko Hayashi: 30/06 Itai...,30/06/2023,YOKO HAYASHI,30/06 ITAIM YOKO ( DISPENSADOPOR MOTIVO DE SA...,CERTO,30/06/2023,ITAIM,1,DISPENSADO,NaN,NaN,NaN,semana,NaN,NaN,NaN,NaN


# Salvando dataframe

In [28]:
dispen[dispen['controle'] == 'CERTO' ].to_excel("dispen_resolvido.xlsx", index=False)

In [29]:
dispen[dispen['controle'] == 'PROB' ].to_excel("dispen_giov.xlsx", index=False)

###Fim Periodo 

In [ ]:
browser.quit()